### 2 Do
* Write over basics

### Done
* Add CartPole env
* Get shit to run
* ReRead tutorial
* Go through code, check TODO's


# Simple Reinforcement Learning with Tensorflow Part 8: Asynchronous Actor-Critic Agents (A3C)
https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-8-asynchronous-actor-critic-agents-a3c-c88f72a5e9f2


Introduction take aways
* The A3C algorithm was released by Google’s DeepMind group earlier this year, and it made a splash by… essentially obsoleting DQN
  * faster, simpler, more robust, and able to achieve much better scores on the standard battery of Deep RL tasks.
  * Continuous + discrete actions
  * New go-to method for complex action + complex state problems


Actor-Critic
* Value function V(s)
* Policy $\pi$(s) (stochastic)
* Advantage: A=R-V(s)
  * We will actually use [**generalized advantage estimation**](https://arxiv.org/abs/1506.02438) (Berkeley)


Code that inspired Juliani
 * [DennyBritz](https://github.com/dennybritz/reinforcement-learning)
 * [OpenAI Starter bot](https://github.com/openai/universe-starter-agent)


## Code break down

**AC_Network**: Creates the networks themselves

**Worker**: Logic for interacting with the environment and updating the global network

For 'fast envionments' (e.g. 60 fps) have one core per worker. For slow environments (e.g. 5 fps) you can have several (e.g. 1-4)

### Entropy
Output: Actions with similar probabilities: **high entropy**

Output: Single action with high probability: **Low entropy**

We **subtract** the entropy (never mind the wierd formulations..) i.e. increase in entropy --> decrease in loss function

In [ ]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run all cells',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.execute_all_cells();
        return false;
    }
});

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
%matplotlib inline

In [ ]:

# import threading
import multiprocessing
import os
# import numpy as np
# import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.contrib.slim as slim
import gym
# import scipy.signal
# from vizdoom import *

# from random import choice
# from time import sleep
# from time import time

import sys
sys.path.append('./..')

from utils import *
from a3c import *
from juliani_helper import *

In [ ]:
## Hyperparameters
env_name = 'Breakout-v0'
env_name = 'CartPole-v1'


max_episode_length = 300
gamma = 0.99
down_sample_factor = 2

load_model = False
model_path = os.path.join('.', 'model')
frames_path = os.path.join('.', 'frames')

In [ ]:
## Setup
num_cpu=multiprocessing.cpu_count()

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(frames_path):
    os.makedirs(frames_path)

env = gym.make(env_name)
observation = env.reset()
# observation = prepro(env.render(mode='rgb_array'), down_sample_factor)
# env.render(close=True)

s_size = list(observation.shape)
a_size = env.action_space.n # number of action dimensions
print('s_size:', s_size)
print('a_size:', a_size)

## Setup tensorboard

In [ ]:
## Print Hyperparameter summary
print('load_model:', load_model)

## Print setup summary
print('Number of available CPU cores:', num_cpu)

In [ ]:
tf.reset_default_graph()

## Initialize worker and agents
with tf.device('/cpu:0'):
    global_episodes = tf.Variable(0, dtype=tf.int32, name='global_episodes',
                                  trainable=False)
    trainer = tf.train.RMSPropOptimizer(learning_rate=1e-3) #TODO: Change to RMSProp
    
    master_network = AC_Network(s_size, a_size, scope='global', trainer=None)
    workers = []
    for i in range(num_cpu):
        env = gym.make(env_name)
        workers.append(Worker(env, i, s_size, a_size, trainer,
                             model_path, global_episodes))
    saver = tf.train.Saver(max_to_keep=5)


## Train model
with tf.Session() as sess:
    coord = tf.train.Coordinator() # Used to coordinate different threads
    
    if load_model:
        print('Loading model...')
        ckpt = tf.rain.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(tf.global_variables_initializer())
    
    worker_threads = []
    for worker in workers:
        worker_work = lambda: worker.work(max_episode_length, gamma, sess, coord, saver)
        t = threading.Thread(target=(worker_work))
        t.start() # set the thread to start working
        sleep(0.5) #TODO: Is this really the best way to do this???!
        worker_threads.append(t)
#         break
    coord.join(worker_threads)

print('Terminated!')

In [ ]:
print('Done')